#### Package Imports

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

#Import Main Data Set
incidents<-read.csv("Use_ofForce.csv",header=T)
BaseDF<-data.frame(Date=incidents[,1])
BaseDF$Month<-as.numeric(regmatches(incidents$Incident.date..dd.mm.yy.,regexpr("\\d*",incidents$Incident.date..dd.mm.yy.)))
BaseDF$Month[BaseDF$Month=="1"]<-"13"
BaseDF$Month[BaseDF$Month=="2"]<-"14"
BaseDF$Hour<-regmatches(incidents$Incident.time..hh.mm.,regexpr("\\d*",incidents$Incident.time..hh.mm.))
BaseDF$Borough<-incidents$LPA.BCU.OPU..UoF.carried.out
BaseDF$Location<-NA
for (i in 1:nrow(incidents)){
  for (j in 3:17){
    if (incidents[i,j]=="Yes"){
      fullLoc<-colnames(incidents)[j]
      BaseDF$Location[i]<-substring(fullLoc, 20)
    }
  }
}
BaseDF$Subject_Behavior<-incidents$Primary..conduct.of.subject...behaviour
BaseDF$Threat_orAssault<-NA
for (i in 1:nrow(incidents)){
  for (j in 20:22){
    if (incidents[i,j]=="Yes"){
      BaseDF$Threat_orAssault[i]<-colnames(incidents)[j]
    }
  }
}
BaseDF[,8:33]<-incidents[,23:48]
BaseDF$Final_Effective_Tactic<-NA
for (i in 1:nrow(incidents)){
  for (j in seq(from=88,to=50,by=-2)){
    if (incidents[i,j]=="Yes"){
      BaseDF$Final_Effective_Tactic[i]<-as.character(incidents[i,j-1])
    }
  }
}
BaseDF$Final_Effective_Tactic<-as.character(BaseDF$Final_Effective_Tactic)
BaseDF$Final_Effective_Tactic[BaseDF$Final_Effective_Tactic=="Unarmed skills (including pressure points, strikes, restraints and take-downs)"]<-"Unarmed Skills"
BaseDF$Final_Effective_Tactic<-as.factor(BaseDF$Final_Effective_Tactic)
BaseDF$Num_Tactics_Tried<-NA
for (i in 1:nrow(incidents)){
  counter<-0
  for (j in seq(from=50,to=88,by=2)){
    if (as.character(incidents[i,j])!="NULL"){
      counter<-counter+1
    }
  }
  BaseDF$Num_Tactics_Tried[i]<-counter
}
BaseDF[,36:54]<-incidents[,253:271]

##################################
# DEMOGRAPHIC AND OTHER METADATA #
##################################
#Add in demographic info by borough
crimeRateDF<-read.csv("crime_rate.csv",header = TRUE, stringsAsFactors = FALSE)
FullDF<-merge(BaseDF,crimeRateDF, all.x = TRUE,by="Borough")

#Add in wellbeing data
wellbeingDF<-read.csv("wellbeing.csv",header=TRUE, stringsAsFactors = FALSE) 
collapse_wellbeing<-aggregate(wellbeingDF[colnames(wellbeingDF)[5:16]], by = list(wellbeingDF$Borough), FUN = "sum")
colnames(collapse_wellbeing)[1]<-"Borough"
FullDF<-merge(FullDF,collapse_wellbeing, all.x = TRUE,by="Borough")

#social metrics
socialDF<-read.csv("social_metrics.csv", stringsAsFactors = FALSE)
FullDF<-merge(FullDF,socialDF, all.x = TRUE,by="Borough")

write.table(FullDF, file="00_FullDF.csv", col.names = TRUE, sep=",")